In [1]:
import pandas as pd
import os
from sksurgerynditracker.nditracker import NDITracker
import numpy as np

In [2]:
"""
    Initialize the NDI Aurora EM tracker
    NOTE: TWO TRACKERS DURING OPERATION
    EM1 for Ultrasound Probe
    EM2 for Patient Body
    tracker.get_frame() will return the tracking results as:
    (
    [10, 11],
    [np.float64(1741808560.8712058), np.float64(1741808560.8712058)],
    [np.float64(3238.0), np.float64(3238.0)],
    [array([[-7.6868730e-01, -6.3685608e-01,  5.5912140e-02,  8.5530000e+01],
            [ 6.3819072e-01, -7.6956820e-01,  8.3178400e-03,  3.6060000e+01],
            [ 3.7741740e-02,  4.2082240e-02,  9.9840086e-01, -1.3366000e+02],
            [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]]),
     array([[ 4.3552278e-01,  8.9886832e-01, -4.6381980e-02,  8.9580000e+01],
            [-8.9901672e-01,  4.3692336e-01,  2.5749920e-02, -1.3790000e+01],
            [ 4.3410620e-02,  3.0493680e-02,  9.9859158e-01, -1.3192000e+02],
            [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])],
    [np.float64(0.060200000000000004), np.float64(0.027200000000000002)]
    )
"""
SETTINGS_TWO_EMs = {
    "tracker type": "aurora",
    "port": "COM3"
}
tracker_AURORA = NDITracker(SETTINGS_TWO_EMs)
tracker_AURORA.start_tracking()

In [ ]:
"""
    Initialize the NDI Polaris Optical tracker (Track Butterfly Plate)
    MAKE SURE you have the Butterfly .rom file and its path
"""
SETTINGS_BF = {
    "tracker type": "vega",
    "ip address": "169.254.7.250",
    "port": 8765,

    # Windows Path
    "romfiles": ["PATH TO BUTTERFLY PLATE .ROM FILE"]
}
tracker_POLARIS = NDITracker(SETTINGS_BF)
tracker_POLARIS.start_tracking()

In [10]:
EM1_to_US = np.load("EM1_to_US.npy") # Fixed relationship, Calculated by US_Aurora_Registration.ipynb
EM2_to_CT = np.load("EM2_to_CT.npy") # Fixed relationship, Calculated by Polaris_Aurora_Registration.ipynb
PO_to_FG = np.load("PO_to_FG.npy")  # Fixed relationship, Calculated by Polaris_Aurora_Registration.ipynb
BF_to_EN = np.eye(4)  # Fixed relationship, between Butterfly and Endoscope Coord. Systems, measured by .stl files

while True:
    # Read new data from Aurora
    EM_handles, EM_timestamps, EM_framenumbers, EM_tracking, EM_quality = tracker_AURORA.get_frame()

    # Read new data from Polaris
    PO_handles, PO_timestamps, PO_framenumbers, PO_tracking, PO_quality = tracker_POLARIS.get_frame()

    # Update EM1 data
    FG_to_EM1_updated = EM_tracking[0]

    # Update EM2 data
    FG_to_EM2_updated = EM_tracking[1]

    # Update Ultrasound Pose
    FG_to_US_updated = np.dot(FG_to_EM1_updated, EM1_to_US)

    # Update CT in Aurora Field Generator
    FG_to_CT_updated = np.dot(FG_to_EM2_updated, EM2_to_CT)

    # Update CT in Polaris
    PO_to_CT_updated = np.dot(PO_to_FG, FG_to_CT_updated)

    # Update Endoscope Butterfly in Polaris
    PO_to_BF = PO_tracking[0]

    # Update Endoscope Coord. System in Polaris
    PO_to_EN = np.dot(PO_to_BF, BF_to_EN)